###IMPORTANTE

Bueno, mi idea aquí es dejar una estructura de cómo sería el código para crear los agentes y resolver mi problema planteado en el ejercicio dos del TP. Obviamente, no tengo API key de OpenAI, pero si la tuviera, cargaría el archivo OAI_CONFIG_LIST a Colab y probaría si funciona  y tampoco tengo pagina ja!

Fuente: https://microsoft.github.io/autogen/docs/Examples/#automated-multi-agent-chat


AutoGen ofrece agentes conversacionales alimentados por LLM, herramientas o humanos, que pueden ser utilizados para realizar tareas de manera colectiva a través de chats automatizados. Este marco permite el uso de herramientas y la participación humana a través de conversaciones de múltiples agentes. Encuentra documentación sobre esta característica aquí.

In [ ]:
!pip install pyautogen


La función config_list_from_json carga una lista de configuraciones desde una variable de entorno o un archivo JSON.

In [ ]:
import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

Primero busca la variable de entorno "OAI_CONFIG_LIST", que debe ser una cadena JSON válida. Si no se encuentra esa variable, entonces busca un archivo JSON llamado "OAI_CONFIG_LIST". Filtra las configuraciones por modelos (también puedes filtrar por otras claves).

La lista de configuraciones se ve como sigue:
```python
config_list = [
    {
        'model': 'gpt-4-32k',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k-0314',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

Puedes establecer el valor de config_list de la manera que prefieras. Por favor, consulta este cuaderno para ver ejemplos completos de código de los diferentes métodos. [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb)

## Realizo los agentes

In [ ]:
gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="""Un administrador humano. Interactúa con todos los agentes para llevar un paso a paso, desde la extracion de texto, el formateo de texto, la traduccion de texto a otros idiomas y por ultimo actulizar la pagina web en cada idioma. La ejecución del plan necesita ser aprobada por este administrador.
        """,
    code_execution_config=False,
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message="""Eres ingeniero experto en procesamiento del lenguaje natural. Escribes el codigo en python, ademas tienes la capacidad para porbar el codigo que produces
    y puede corregir erroes. Sos especialista en extraer texto de paginas web
""",
)
engineer1 = autogen.AssistantAgent(
    name="Engineer1",
    llm_config=gpt4_config,
    system_message="""Eres ingeniero experto en procesamiento del lenguaje natural. Escribes el codigo en python, ademas tienes la capacidad para porbar el codigo que produces
    y puede corregir erroes. Sos especialista en formatear texto.
""",
)
marketing_digital = autogen.AssistantAgent(
    name="marketing_digital",
    llm_config=gpt4_config,
    system_message="""Estas capacitado en diversas áreas del marketing digital, incluida la creación y optimización de páginas web de publicidad, el diseño de estrategias de marketing en línea, la gestión de campañas publicitarias en plataformas digitales, el análisis de datos y métricas para medir el rendimiento de las campañas.""",
)
traductor = autogen.AssistantAgent(
    name="traductor",
    system_message="""Eres un experto traductor de inglés. Tu tarea principal es traducir textos del español al ingles con precisión y fluidez. Estás familiarizado con una amplia variedad de contextos y terminologías en inglés, y puedes adaptar tus traducciones según sea necesario para garantizar la máxima comprensión y calidad. Además, estás disponible para realizar revisiones y ajustes en las traducciones según las solicitudes del usuario
""",
    llm_config=gpt4_config,
)
traductor1 = autogen.AssistantAgent(
    name="traductor",
    system_message="""Eres un experto traductor de italiano. Tu tarea principal es traducir textos del español al italiano con precisión y fluidez. Estás familiarizado con una amplia variedad de contextos y terminologías en inglés, y puedes adaptar tus traducciones según sea necesario para garantizar la máxima comprensión y calidad. Además, estás disponible para realizar revisiones y ajustes en las traducciones según las solicitudes del usuario
""",
    llm_config=gpt4_config,
)
traductor2 = autogen.AssistantAgent(
    name="traductor2",
    system_message="""Eres un experto traductor de Frances. Tu tarea principal es traducir textos del español al frances con precisión y fluidez. Estás familiarizado con una amplia variedad de contextos y terminologías en inglés, y puedes adaptar tus traducciones según sea necesario para garantizar la máxima comprensión y calidad. Además, estás disponible para realizar revisiones y ajustes en las traducciones según las solicitudes del usuario
""",
    llm_config=gpt4_config,
)
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, engineer1, marketing_digital, traductor, traductor1, traductor2], messages=[], max_round=50
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

##Iniciamos el chat

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="""
Traducir la pagina de español a los demas idiomas, este es el URL:.....
""",
)